In [21]:
from osgeo import gdal
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist, euclidean
import math


#### Coordinate transformation

In [ ]:
# xy->XY
def xy2XY(x, y, geo):
    X = geo[0] + x * geo[1] + y * geo[2]
    Y = geo[3] + x * geo[4] + y * geo[5]
    return X, Y


def xy2XY_2(xy, geo):
    X = geo[0] + xy[0] * geo[1] + xy[1] * geo[2]
    Y = geo[3] + xy[0] * geo[4] + xy[1] * geo[5]
    return X, Y

# XY->xy
def XY2xy(X, Y, geo):
    a = np.array([[geo[1], geo[2]], [geo[4], geo[5]]])
    b = np.array([X - geo[0], Y - geo[3]])
    x, y = np.linalg.solve(a, b)
    return x, y


# Longitude error of TPs
def Xdistance(geo1, geo2, x1, x2, y1, y2):
    dX = geo1[0] + geo1[1] * x1 + geo1[2] * y1 - geo2[0] - geo2[1] * x2 - geo2[2] * y2
    return dX


# Latitude error of TPs
def Ydistance(geo1, geo2, x1, x2, y1, y2):
    dY = geo1[3] + geo1[4] * x1 + geo1[5] * y1 - geo2[3] - geo2[4] * x2 - geo2[5] * y2
    return dY

#### Relative Horizontal Correction Methods

##### Least Square Method

In [ ]:
# Least Square Method with perturbation，LSM_p1
def adjustment_LSM_p1(TP_xy, g1, g2):
    geo1 = np.copy(g1)  # Deep copy
    geo2 = np.copy(g2)
    TP_num = TP_xy.shape[1]

    b = []
    for i in range(0, TP_num):
        b.append(-Xdistance(geo1, geo2, TP_xy[0, i], TP_xy[2, i], TP_xy[1, i], TP_xy[3, i]))
    for i in range(0, TP_num):
        b.append(-Ydistance(geo1, geo2, TP_xy[0, i], TP_xy[2, i], TP_xy[1, i], TP_xy[3, i]))
    b = np.array(b)

    mat0 = np.ones((TP_num, 1))
    mat1 = np.random.random((TP_num, 1)) / 10 - 0.05 + mat0  
    '''
    The disturbance should not be too small; otherwise, it will lead to a pathological system of equations.
    '''
    mat2 = -1 * mat0
    mat3 = np.zeros((TP_num, 2))
    A = np.bmat("mat1 mat2 mat3;mat3 mat1 mat2")

    dXY = np.linalg.inv(A.T @ A) @ A.T @ b

    geo1 = list(geo1)
    geo2 = list(geo2)
    geo1[0] -= dXY[0, 1]
    geo1[3] -= dXY[0, 3]
    geo2[0] -= dXY[0, 0]
    geo2[3] -= dXY[0, 2]
    return [-dXY[0, 1] + dXY[0, 0], -dXY[0, 3] + dXY[0, 2]], geo1, geo2


# Least Square Method with perturbation，LSM_p2
def adjustment_LSM_p2(TP_xy, g1, g2):
    geo1 = np.copy(g1)  
    geo2 = np.copy(g2)
    TP_num = TP_xy.shape[1]

    b = []
    for i in range(0, TP_num):
        b.append(-Xdistance(geo1, geo2, TP_xy[0, i], TP_xy[2, i], TP_xy[1, i], TP_xy[3, i]))
    for i in range(0, TP_num):
        b.append(-Ydistance(geo1, geo2, TP_xy[0, i], TP_xy[2, i], TP_xy[1, i], TP_xy[3, i]))
    b = np.array(b)

    mat0 = np.ones((TP_num, 1))
    mat1 = mat0
    mat2 = -1 * mat0
    mat3 = np.zeros((TP_num, 2))
    A = np.bmat("mat1 mat2 mat3;mat3 mat1 mat2")

    dXY = np.linalg.pinv(A.T @ A) @ A.T @ b

    geo1 = list(geo1)
    geo2 = list(geo2)
    geo1[0] -= dXY[0, 1]
    geo1[3] -= dXY[0, 3]
    geo2[0] -= dXY[0, 0]
    geo2[3] -= dXY[0, 2]
    return [-dXY[0, 1] + dXY[0, 0], -dXY[0, 3] + dXY[0, 2]], geo1, geo2


##### Least Absolute Deviations, LAD

One-dimensional

In [ ]:
def adjustment_LAD(TP_xy, g1, g2):
    geo1 = np.copy(g1)  
    geo2 = np.copy(g2)
    TP_num = TP_xy.shape[1]

    Xerror = []
    Yerror = []
    for i in range(0, TP_num):
        Xerror.append(-Xdistance(geo1, geo2, TP_xy[0, i], TP_xy[2, i], TP_xy[1, i], TP_xy[3, i]))
        Yerror.append(-Ydistance(geo1, geo2, TP_xy[0, i], TP_xy[2, i], TP_xy[1, i], TP_xy[3, i]))
    Xerror.sort()
    Yerror.sort()

    if TP_num % 2 == 0:
        '''
        When there are an even number of samples,take the mean of the middle two.
        For example,if there are 4 samples, the indexes of the middle two samples are 1 and 2.
        '''
        dX = (Xerror[TP_num // 2] + Xerror[TP_num // 2 - 1]) / 2
        dY = (Yerror[TP_num // 2] + Yerror[TP_num // 2 - 1]) / 2
    else:
        '''
        When there are an odd number of samples, take the middle sample. 
        For example, if there are 5 samples, the index of the middle sample is 2
        '''
        dX = Xerror[(TP_num - 1) // 2]
        dY = Yerror[(TP_num - 1) // 2]

    geo1[0] += dX/2
    geo1[3] += dY/2
    geo2[0] -= dX/2
    geo2[3] -= dY/2
    return [dX, dY], geo1, geo2


Two-dimensional

In [ ]:
def l1_median(X, eps=1e-6):
    """
    Computes weighted geometric median
    :param X: the list of sample points, a 2D ndarray
    :param eps: acceptable error margin
    :return: first estimate meeting eps
    """
    y = np.mean(X, 0)  # the geometric mean is a fare start
    while True:
        while np.any(cdist(X, [y]) == 0):  # Euclidean distances, let's move away to avoid any null
            y += 0.1 * np.ones(len(y))
        # set of weights that are the inverse of the distances from current estimate to the observations
        W = 1 / cdist(X, [y])  # element-wise
        # new estimate is the weighted average of the observations
        y1 = np.sum(W * X, 0) / np.sum(W)  # sum along axis 0
        if euclidean(y, y1) < eps:
            return y1
        y = y1


def adjustment_LAD_XY(TP_xy, g1, g2):
    geo1 = np.copy(g1)  
    geo2 = np.copy(g2)
    TP_num = TP_xy.shape[1]

    Xerror = []
    Yerror = []
    for i in range(0, TP_num):
        Xerror.append(-Xdistance(geo1, geo2, TP_xy[0, i], TP_xy[2, i], TP_xy[1, i], TP_xy[3, i]))
        Yerror.append(-Ydistance(geo1, geo2, TP_xy[0, i], TP_xy[2, i], TP_xy[1, i], TP_xy[3, i]))
    points = np.array([Xerror, Yerror])
    points = points.T
    distance = l1_median(points)

    geo1[0] += distance[0]/2
    geo1[3] += distance[1]/2
    geo2[0] -= distance[0]/2
    geo2[3] -= distance[1]/2
    return [distance[0],distance[1]], geo1, geo2


##### Least Square Method,LSM

In [ ]:
# One-dimensional / Two-dimensional  (the same)
def adjustment_LSM(TP_xy, g1, g2):
    geo1 = np.copy(g1)  
    geo2 = np.copy(g2)
    TP_num = TP_xy.shape[1]

    Xerror = []
    Yerror = []
    for i in range(0, TP_num):
        Xerror.append(Xdistance(geo1, geo2, TP_xy[0, i], TP_xy[2, i], TP_xy[1, i], TP_xy[3, i]))
        Yerror.append(Ydistance(geo1, geo2, TP_xy[0, i], TP_xy[2, i], TP_xy[1, i], TP_xy[3, i]))
    Xerror = np.array(Xerror)
    Yerror = np.array(Yerror)

    dX = Xerror.mean()
    dY = Yerror.mean()

    geo1[0] -= dX/2
    geo1[3] -= dY/2
    geo2[0] += dX/2
    geo2[3] += dY/2
    return [-dX, -dY], geo1, geo2


##### Point increment method to find the minimum covering circle

In [ ]:
def distance(A, B):
    return math.sqrt((A[0] - B[0]) ** 2 + (A[1] - B[1]) ** 2)


def circumcircle(A, B, C):
    # Prevent the situation where three points are in a straight line
    eps = 0.000000000000000000001

    a = distance(A, B)
    b = distance(B, C)
    c = distance(C, A)

    r = (a * b * c) / math.sqrt((a + b + c) * (-a + b + c) * (a - b + c) * (a + b - c) + eps)

    # Calculate the circumcenter coordinates
    a1 = 2 * (A[0] - B[0])
    b1 = 2 * (A[1] - B[1])
    a2 = 2 * (B[0] - C[0])
    b2 = 2 * (B[1] - C[1])
    c1 = A[0] ** 2 - B[0] ** 2 + A[1] ** 2 - B[1] ** 2
    c2 = B[0] ** 2 - C[0] ** 2 + B[1] ** 2 - C[1] ** 2

    x = (b2 * c1 - b1 * c2) / (a1 * b2 - a2 * b1 + eps)
    y = (-a2 * c1 + a1 * c2) / (a1 * b2 - a2 * b1 + eps)
    circum_point = np.array([x, y])

    results = [r, circum_point]

    return results


def twoPointCircle(A, B, C):
    centerPoint = np.array([(A[0] + B[0]) / 2, (A[1] + B[1]) / 2])
    r = distance(A, B) / 2

    d = distance(C, centerPoint)

    if d <= r:
        results = [r, centerPoint]
    else:
        results = [-1, centerPoint]

    return results


In [ ]:
# Find the smallest covering circle among the three points
def minCircle(A, B, C):
    # Circumscribed circle
    circum_results = circumcircle(A, B, C)

    # A circle with two of its diameters
    AB_results = twoPointCircle(A, B, C)
    BC_results = twoPointCircle(B, C, A)
    CA_results = twoPointCircle(C, A, B)

    circleOfTwoPoint = [AB_results, BC_results, CA_results]

    min_r = circum_results[0]
    min_circle_point = circum_results[1]

    for result in circleOfTwoPoint:
        if result[0] == -1:
            continue
        else:
            if result[0] < min_r:
                min_r = result[0]
                min_circle_point = result[1]

    return min_r, min_circle_point


# Find point D in the point set that is the smallest covering circle away from points A, B and C
def findLongestPoint(pointSet, centralPoint):
    d = 0
    longestPoint = None

    for p in pointSet:
        d_p = distance(p, centralPoint)
        if d_p >= d:
            longestPoint = p
            d = d_p

    return longestPoint


def isInMinCircle(A, B, C, D):
    min_r, min_circle_point = minCircle(A, B, C)

    d = distance(D, min_circle_point)

    if d > min_r:
        results = [0, min_r, A, B, C]
    else:
        results = [1, min_r, A, B, C]

    return results


def findNewThreePoint(A, B, C, D):
    ABD_results = isInMinCircle(A, B, D, C)
    ACD_results = isInMinCircle(A, C, D, B)
    BCD_results = isInMinCircle(B, C, D, A)

    results = [ABD_results, ACD_results, BCD_results]

    min_r = 100000000

    for result in results:
        if result[0] == 0:
            continue
        else:
            if result[1] <= min_r:
                A = result[2]
                B = result[3]
                C = result[4]

    return A, B, C


In [29]:
def plt_circle(all_p, three_p, min_circle_point, min_r):
    figure, axes = plt.subplots()
    draw_circle = plt.Circle(min_circle_point, min_r, fill=False)
    axes.set_aspect(1)
    axes.add_artist(draw_circle)

    plt.scatter(all_p[:, 0], all_p[:, 1])
    three_p = np.array(three_p)
    plt.scatter(three_p[:, 0], three_p[:, 1])
    plt.scatter(min_circle_point[0], min_circle_point[1])

    pad_size = 0.0003
    plt.xlim(min(all_p[:, 0]) - pad_size, max(all_p[:, 0]) + pad_size)
    plt.ylim(min(all_p[:, 1]) - pad_size, max(all_p[:, 1]) + pad_size)

    plt.show()


In [30]:
def min_cover_circle(pointSet, plt, eps=1e-6):
    A = pointSet[0]
    B = pointSet[1]
    C = pointSet[2]
    min_r, min_circle_point = minCircle(A, B, C)
    if plt:
        plt_circle(pointSet, [A, B, C], min_circle_point, min_r)
    D = findLongestPoint(pointSet, min_circle_point)

    while distance(D, min_circle_point) - min_r > eps:
        A, B, C = findNewThreePoint(A, B, C, D)
        min_r, min_circle_point = minCircle(A, B, C)
        if plt:
            plt_circle(pointSet, [A, B, C], min_circle_point, min_r)

        D = findLongestPoint(pointSet, min_circle_point)
    return -min_circle_point


##### Min-max Absolute Deviations,MAD

One-dimensional

In [ ]:
def adjustment_MAD(TP_xy, g1, g2):
    geo1 = np.copy(g1)  
    geo2 = np.copy(g2)
    TP_num = TP_xy.shape[1]

    Xerror = []
    Yerror = []
    for i in range(0, TP_num):
        Xerror.append(Xdistance(geo1, geo2, TP_xy[0, i], TP_xy[2, i], TP_xy[1, i], TP_xy[3, i]))
        Yerror.append(Ydistance(geo1, geo2, TP_xy[0, i], TP_xy[2, i], TP_xy[1, i], TP_xy[3, i]))
    Xerror.sort()
    Yerror.sort()

    dX = (Xerror[0] + Xerror[TP_num - 1]) / 2
    dY = (Yerror[0] + Yerror[TP_num - 1]) / 2

    geo1[0] -= dX/2
    geo1[3] -= dY/2
    geo2[0] += dX/2
    geo2[3] += dY/2
    return [-dX, -dY], geo1, geo2


Two-dimensional

In [ ]:
def adjustment_MAD_XY(TP_xy, g1, g2, plt=0):
    geo1 = np.copy(g1)  
    geo2 = np.copy(g2)
    TP_num = TP_xy.shape[1]

    Xerror = []
    Yerror = []
    for i in range(0, TP_num):
        Xerror.append(Xdistance(geo1, geo2, TP_xy[0, i], TP_xy[2, i], TP_xy[1, i], TP_xy[3, i]))
        Yerror.append(Ydistance(geo1, geo2, TP_xy[0, i], TP_xy[2, i], TP_xy[1, i], TP_xy[3, i]))
    points = np.array([Xerror, Yerror])
    points = points.T
    distance = min_cover_circle(points, plt)

    geo1[0] += distance[0]/2
    geo1[3] += distance[1]/2
    geo2[0] -= distance[0]/2
    geo2[3] -= distance[1]/2
    return distance, geo1, geo2


#### Error analysis

In [ ]:
# Unit conversion: Longitude and latitude difference to meters
def Haversine(lon1, lon2, lat1, lat2):
    dlon = lon1 - lon2
    dlat = lat1 - lat2
    a = pow(math.sin(math.radians(dlat / 2)), 2) + math.cos(math.radians(lat1)) * math.cos(
        math.radians(lat1)
    ) * pow(math.sin(math.radians(dlon / 2)), 2)
    d = 2 * 6378000 * math.asin(math.sqrt(a))
    if dlon < 0 and dlat == 0:
        d = -d
    if dlon == 0 and dlat < 0:
        d = -d
    return d


In [ ]:
# Calculation of TPs error
def calculate_plane_error(TP_xy, geo1, geo2, method):
    TP_num = TP_xy.shape[1]
    error_X = []
    error_Y = []
    error_XY = []
    abs_error_X = []
    abs_error_Y = []
    sq_error_X = []
    sq_error_Y = []
    sq_error_XY = []
    if method == "Degree":
        TP_X1, TP_Y1 = xy2XY(TP_xy[0, :], TP_xy[1, :], geo1)
        TP_X2, TP_Y2 = xy2XY(TP_xy[2, :], TP_xy[3, :], geo2)
        for i in range(0, TP_num):
            error_X.append(TP_X1[i] - TP_X2[i])
            abs_error_X.append(abs(error_X[i]))
            sq_error_X.append(error_X[i] ** 2)

            error_Y.append(TP_Y1[i] - TP_Y2[i])
            abs_error_Y.append(abs(error_Y[i]))
            sq_error_Y.append(error_Y[i] ** 2)

            error_XY.append(pow(error_X[i] ** 2 + error_Y[i] ** 2, 0.5))
            sq_error_XY.append(error_XY[i] ** 2)

    if method == "Haversine":
        TP_X1, TP_Y1 = xy2XY(TP_xy[0, :], TP_xy[1, :], geo1)
        TP_X2, TP_Y2 = xy2XY(TP_xy[2, :], TP_xy[3, :], geo2)
        for i in range(0, TP_num):
            error_X.append(
                Haversine(TP_X1[i], TP_X2[i], (TP_Y1[i] + TP_Y2[i]) / 2, (TP_Y1[i] + TP_Y2[i]) / 2)
            )
            abs_error_X.append(abs(error_X[i]))
            sq_error_X.append(error_X[i] ** 2)

            error_Y.append(
                Haversine((TP_X1[i] + TP_X2[i]) / 2, (TP_X1[i] + TP_X2[i]) / 2, TP_Y1[i], TP_Y2[i])
            )
            abs_error_Y.append(abs(error_Y[i]))
            sq_error_Y.append(error_Y[i] ** 2)

            error_XY.append(Haversine(TP_X1[i], TP_X2[i], TP_Y1[i], TP_Y2[i]))
            sq_error_XY.append(error_XY[i] ** 2)

    if method == "Geo":
        meter_resolution = 30
        for i in range(0, TP_num):
            X_temp, Y_temp = xy2XY(TP_xy[2, i], TP_xy[3, i], geo2)
            x_temp, y_temp = XY2xy(X_temp, Y_temp, geo1)

            error_X.append((TP_xy[0, i] - x_temp) * meter_resolution)
            abs_error_X.append(abs(error_X[i]))
            sq_error_X.append(error_X[i] ** 2)

            error_Y.append((TP_xy[1, i] - y_temp) * meter_resolution)
            abs_error_Y.append(abs(error_Y[i]))
            sq_error_Y.append(error_Y[i] ** 2)

            error_XY.append(pow(error_X[i] ** 2 + error_Y[i] ** 2, 0.5))
            sq_error_XY.append(error_XY[i] ** 2)

    error_X.sort()
    abs_error_X.sort()
    error_Y.sort()
    abs_error_Y.sort()
    error_XY.sort()

    mean = [
        sum(error_X) / TP_num,
        sum(error_Y) / TP_num,
        sum(error_XY) / TP_num,
    ]
    mean_abs = [
        sum(abs_error_X) / TP_num,
        sum(abs_error_Y) / TP_num,
        sum(error_XY) / TP_num,
    ]
    mid_abs = [
        abs_error_X[int(TP_num * 0.5)],
        abs_error_Y[int(TP_num * 0.5)],
        error_XY[int(TP_num * 0.5)],
    ]
    max_abs = [
        abs_error_X[TP_num - 1],
        abs_error_Y[TP_num - 1],
        error_XY[TP_num - 1],
    ]
    CE90_abs = [
        abs_error_X[int(TP_num * 0.9)],
        abs_error_Y[int(TP_num * 0.9)],
        error_XY[int(TP_num * 0.9)],
    ]
    RMSE = [
        pow(sum(sq_error_X) / TP_num, 0.5),
        pow(sum(sq_error_Y) / TP_num, 0.5),
        pow(sum(sq_error_XY) / TP_num, 0.5),
    ]
    return mean, mean_abs, mid_abs, max_abs, CE90_abs, RMSE, error_X, error_Y, error_XY


### Set input and output

In [ ]:
# The number of pairs of images to be adjusted
IMG_pair_num = 7

# Error calculation method("Degree","Haversine","Geo")
METHOD = "Geo"

# Import satellite image positioning parameters
GEOs = np.load("dataset/dem_info/GEOs.npy")


In [36]:
GEOs_LSM_p1 = []
GEOs_LSM_p2 = []
GEOs_LAD = []
GEOs_LAD_XY = []
GEOs_LSM = []
GEOs_MAD = []
GEOs_MAD_XY = []

Distance_LSM_p1 = []
Distance_LSM_p2 = []
Distance_LAD = []
Distance_LAD_XY = []
Distance_LSM = []
Distance_MAD = []
Distance_MAD_XY = []


### Relative Horizontal Correction

In [ ]:
# Select the image pair number
for Select_IMGpair in range(0, IMG_pair_num):
    print(" ")
    print("The {} pair".format(Select_IMGpair))

    # Read data
    geo1_origin = np.copy(GEOs[Select_IMGpair*2])
    geo2_origin = np.copy(GEOs[Select_IMGpair*2+1])
    temp_TPs = np.load("dataset/TP/" + str(Select_IMGpair) + ".npy")
    print("TPs num:{}".format(temp_TPs.shape[1]))

    # Raw data
    _, mean_temp, _, max_temp, _, RMSE_temp, _, _, _ = calculate_plane_error(
        temp_TPs, geo1_origin, geo2_origin, METHOD
    )
    print("Raw img")
    print("MEAN:X{},Y direction{},XY direction{}".format(mean_temp[0], mean_temp[1], mean_temp[2]))
    print("RMSE:X direction{},Y direction{},XY direction{}".format(RMSE_temp[0], RMSE_temp[1], RMSE_temp[2]))
    print("MAX:X direction{},Y direction{},XY direction{}".format(max_temp[0], max_temp[1], max_temp[2]))

    # LSM_p1
    Distance_temp, geo1_LSM_p1, geo2_LSM_p1 = adjustment_LSM_p1(temp_TPs, geo1_origin, geo2_origin)
    _, mean_temp, _, max_temp, _, RMSE_temp, _, _, _ = calculate_plane_error(
        temp_TPs, geo1_LSM_p1, geo2_LSM_p1, METHOD
    )
    print("LSM_p1")
    print("Distance{}".format(Distance_temp))
    print("MEAN:X direction{},Y direction{},XY direction{}".format(mean_temp[0], mean_temp[1], mean_temp[2]))
    print("RMSE:X direction{},Y direction{},XY direction{}".format(RMSE_temp[0], RMSE_temp[1], RMSE_temp[2]))
    print("MAX:X direction{},Y direction{},XY direction{}".format(max_temp[0], max_temp[1], max_temp[2]))

    GEOs_LSM_p1.append(geo1_LSM_p1)
    GEOs_LSM_p1.append(geo2_LSM_p1)
    Distance_LSM_p1.append(Distance_temp)

    # LSM_p2
    Distance_temp, geo1_LSM_p2, geo2_LSM_p2 = adjustment_LSM_p2(temp_TPs, geo1_origin, geo2_origin)
    _, mean_temp, _, max_temp, _, RMSE_temp, _, _, _ = calculate_plane_error(
        temp_TPs, geo1_LSM_p2, geo2_LSM_p2, METHOD
    )
    print("LSM_p2")
    print("Distance{}".format(Distance_temp))
    print("MEAN:X direction{},Y direction{},XY direction{}".format(mean_temp[0], mean_temp[1], mean_temp[2]))
    print("RMSE:X direction{},Y direction{},XY direction{}".format(RMSE_temp[0], RMSE_temp[1], RMSE_temp[2]))
    print("MAX:X direction{},Y direction{},XY direction{}".format(max_temp[0], max_temp[1], max_temp[2]))

    GEOs_LSM_p2.append(geo1_LSM_p2)
    GEOs_LSM_p2.append(geo2_LSM_p2)
    Distance_LSM_p2.append(Distance_temp)

    # LAD
    Distance_temp, geo1_LAD, geo2_LAD = adjustment_LAD(temp_TPs, geo1_origin, geo2_origin)
    _, mean_temp, _, max_temp, _, RMSE_temp, _, _, _ = calculate_plane_error(
        temp_TPs, geo1_LAD, geo2_LAD, METHOD
    )
    print("LAD")
    print("Distance{}".format(Distance_temp))
    print("MEAN:X direction{},Y direction{},XY direction{}".format(mean_temp[0], mean_temp[1], mean_temp[2]))
    print("RMSE:X direction{},Y direction{},XY direction{}".format(RMSE_temp[0], RMSE_temp[1], RMSE_temp[2]))
    print("MAX:X direction{},Y direction{},XY direction{}".format(max_temp[0], max_temp[1], max_temp[2]))

    GEOs_LAD.append(geo1_LAD)
    GEOs_LAD.append(geo2_LAD)
    Distance_LAD.append(Distance_temp)

    # LAD_XY
    Distance_temp, geo1_LAD_XY, geo2_LAD_XY = adjustment_LAD_XY(temp_TPs, geo1_origin, geo2_origin)
    _, mean_temp, _, max_temp, _, RMSE_temp, _, _, _ = calculate_plane_error(
        temp_TPs, geo1_LAD_XY, geo2_LAD_XY, METHOD
    )
    print("LAD_XY")
    print("Distance{}".format(Distance_temp))
    print("MEAN:X direction{},Y direction{},XY direction{}".format(mean_temp[0], mean_temp[1], mean_temp[2]))
    print("RMSE:X direction{},Y direction{},XY direction{}".format(RMSE_temp[0], RMSE_temp[1], RMSE_temp[2]))
    print("MAX:X direction{},Y direction{},XY direction{}".format(max_temp[0], max_temp[1], max_temp[2]))

    GEOs_LAD_XY.append(geo1_LAD_XY)
    GEOs_LAD_XY.append(geo2_LAD_XY)
    Distance_LAD_XY.append(Distance_temp)

    # LSM
    Distance_temp, geo1_LSM, geo2_LSM = adjustment_LSM(temp_TPs, geo1_origin, geo2_origin)
    _, mean_temp, _, max_temp, _, RMSE_temp, _, _, _ = calculate_plane_error(
        temp_TPs, geo1_LSM, geo2_LSM, METHOD
    )
    print("LSM")
    print("Distance{}".format(Distance_temp))
    print("MEAN:X direction{},Y direction{},XY direction{}".format(mean_temp[0], mean_temp[1], mean_temp[2]))
    print("RMSE:X direction{},Y direction{},XY direction{}".format(RMSE_temp[0], RMSE_temp[1], RMSE_temp[2]))
    print("MAX:X direction{},Y direction{},XY direction{}".format(max_temp[0], max_temp[1], max_temp[2]))

    GEOs_LSM.append(geo1_LSM)
    GEOs_LSM.append(geo2_LSM)
    Distance_LSM.append(Distance_temp)

    # MAD
    Distance_temp, geo1_MAD, geo2_MAD = adjustment_MAD(temp_TPs, geo1_origin, geo2_origin)
    _, mean_temp, _, max_temp, _, RMSE_temp, _, _, _ = calculate_plane_error(
        temp_TPs, geo1_MAD, geo2_MAD, METHOD
    )
    print("MAD")
    print("Distance{}".format(Distance_temp))
    print("MEAN:X direction{},Y direction{},XY direction{}".format(mean_temp[0], mean_temp[1], mean_temp[2]))
    print("RMSE:X direction{},Y direction{},XY direction{}".format(RMSE_temp[0], RMSE_temp[1], RMSE_temp[2]))
    print("MAX:X direction{},Y direction{},XY direction{}".format(max_temp[0], max_temp[1], max_temp[2]))

    GEOs_MAD.append(geo1_MAD)
    GEOs_MAD.append(geo2_MAD)
    Distance_MAD.append(Distance_temp)

    # MAD_XY
    Distance_temp, geo1_MAD_XY, geo2_MAD_XY = adjustment_MAD_XY(temp_TPs, geo1_origin, geo2_origin)
    _, mean_temp, _, max_temp, _, RMSE_temp, _, _, _ = calculate_plane_error(
        temp_TPs, geo1_MAD_XY, geo2_MAD_XY, METHOD
    )
    print("MAD_XY")
    print("Distance{}".format(Distance_temp))
    print("MEAN:X direction{},Y direction{},XY direction{}".format(mean_temp[0], mean_temp[1], mean_temp[2]))
    print("RMSE:X direction{},Y direction{},XY direction{}".format(RMSE_temp[0], RMSE_temp[1], RMSE_temp[2]))
    print("MAX:X direction{},Y direction{},XY direction{}".format(max_temp[0], max_temp[1], max_temp[2]))

    GEOs_MAD_XY.append(geo1_MAD_XY)
    GEOs_MAD_XY.append(geo2_MAD_XY)
    Distance_MAD_XY.append(Distance_temp)


### Save output

In [ ]:
# # Save the positioning parameters after relative adjustment

# np.save("dataset/PA1/GEOs_LSM_p1", GEOs_LSM_p1)
# np.save("dataset/PA1/GEOs_LSM_p2", GEOs_LSM_p2)
# np.save("dataset/PA1/GEOs_LAD", GEOs_LAD)
# np.save("dataset/PA1/GEOs_LAD_XY", GEOs_LAD_XY)
# np.save("dataset/PA1/GEOs_LSM", GEOs_LSM)
# np.save("dataset/PA1/GEOs_MAD", GEOs_MAD)
# np.save("dataset/PA1/GEOs_MAD_XY", GEOs_MAD_XY)


In [ ]:
# # Save the relative distance

# Distance_LSM_p1 = np.array(Distance_LSM_p1)
# Distance_LSM_p2 = np.array(Distance_LSM_p2)
# Distance_LAD = np.array(Distance_LAD)
# Distance_LAD_XY = np.array(Distance_LAD_XY)
# Distance_LSM = np.array(Distance_LSM)
# Distance_MAD = np.array(Distance_MAD)
# Distance_MAD_XY = np.array(Distance_MAD_XY)

# np.save("dataset/PA1/Distance_LSM_p1", Distance_LSM_p1)
# np.save("dataset/PA1/Distance_LSM_p2", Distance_LSM_p2)
# np.save("dataset/PA1/Distance_LAD", Distance_LAD)
# np.save("dataset/PA1/Distance_LAD_XY", Distance_LAD_XY)
# np.save("dataset/PA1/Distance_LSM", Distance_LSM)
# np.save("dataset/PA1/Distance_MAD", Distance_MAD)
# np.save("dataset/PA1/Distance_MAD_XY", Distance_MAD_XY)


In [ ]:
# # Export the relative distance of the image pairs as Excel

# col1 = "LSM_p1_X"
# col2 = "LSM_p1_Y"
# col3 = "LSM_p2_X"
# col4 = "LSM_p2_Y"
# col5 = "LAD_X"
# col6 = "LAD_Y"
# col7 = "LAD_XY_X"
# col8 = "LAD_XY_Y"
# col9 = "LSM_X"
# col10 = "LSM_Y"
# col11 = "MAD_X"
# col12 = "MAD_Y"
# col13 = "MAD_XY_X"
# col14 = "MAD_XY_Y"


# data = pd.DataFrame(
#     {

#         col1: Distance_LSM_p1[:, 0],
#         col2: Distance_LSM_p1[:, 1],
#         col3: Distance_LSM_p2[:, 0],
#         col4: Distance_LSM_p2[:, 1],
#         col5: Distance_LAD[:, 0],
#         col6: Distance_LAD[:, 1],
#         col7: Distance_LAD_XY[:, 0],
#         col8: Distance_LAD_XY[:, 1],
#         col9: Distance_LSM[:, 0],
#         col10: Distance_LSM[:, 1],
#         col11: Distance_MAD[:, 0],
#         col12: Distance_MAD[:, 1],
#         col13: Distance_MAD_XY[:, 0],
#         col14: Distance_MAD_XY[:, 1],
#     }
# )

# data.to_excel("output/PA1/Plane_Distance.xlsx", sheet_name="sheet1", index=False)
